## Neural Nets With Training
This is all the same as before...

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            # Try sigmoid!
            #nn.Sigmoid(),
            nn.Linear(512, 512),
            nn.ReLU(),
            #nn.Sigmoid(),
            nn.Linear(512, 10),
            nn.ReLU(),
            #nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Set hyperparameters

In [3]:
learning_rate = 0.1
batch_size = 32
epochs = 3

Loss Function - cross entropy loss

In [5]:
loss_fn = nn.CrossEntropyLoss()

SGD - Stochastic Gradient Descent. This is a method for adjusting our model parameters to minimize loss.

Which direction increases loss the most? Change parameters in the **opposite** direction.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Now create training loop and test loop functions that set parameters + test our model! NOTES: 
- optimizer.zero_grad sets the gradients back to zero. We need to do this because otherwise the gradients add up, we don't want this!
- loss.backwards backpropogates gradients (i.e. calculate gradients for each relevant variable)
- optimizer.step() adjusts the model parameters

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299820  [    0/60000]
loss: 1.684995  [ 6400/60000]
loss: 1.593637  [12800/60000]
loss: 1.789186  [19200/60000]
loss: 1.408015  [25600/60000]
loss: 1.633760  [32000/60000]
loss: 1.615539  [38400/60000]
loss: 1.633651  [44800/60000]
loss: 1.666712  [51200/60000]
loss: 1.813366  [57600/60000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 0.022017 

Epoch 2
-------------------------------
loss: 1.388217  [    0/60000]
loss: 1.255281  [ 6400/60000]
loss: 1.383880  [12800/60000]
loss: 1.439965  [19200/60000]
loss: 1.107755  [25600/60000]
loss: 1.425321  [32000/60000]
loss: 1.199127  [38400/60000]
loss: 1.405067  [44800/60000]
loss: 1.358490  [51200/60000]
loss: 1.384553  [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 0.020901 

Epoch 3
-------------------------------
loss: 1.314806  [    0/60000]
loss: 1.190368  [ 6400/60000]
loss: 1.267247  [12800/60000]
loss: 1.349691  [19200/60000]
loss: 0.986404  [25600/60000]
loss: 1.282620  [32000/600

Save the trained model into another file

In [7]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
